# Introduction

# Import some packages that we are going to use
If you do not have any of them install it by using pip/ pip3

In [5]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from random import randint
from tqdm import tqdm

# Start web browser using Selenium

By using Selenium we can basically imitate that we are using a proper web browser and browsing like that on a website. It is useful since with other methods not every element of the webpage is loaded.

We can also set some preferences that speeds up our web scraping. In this case since I do not want to scrape images, so our browser will not load them.

This is the most easiest way to start Selenium. As you will see it it will open a Chrome window:

In [2]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
#tes



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-2-6baf748b4601>:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


# Scraping

I will show you how to scrape a website using www.mobile.de which is the leading secondhand (and new) vehicle marketplace. Mobile.de is from Germany and since the German economy is the biggest in Europe and also they car manufacturing is one of their main industry (think about Volkswagen, BMW, Mercedes and Audi) the site is one of the biggest car listing website in the World. It contains millions of advertisements.

My approach to scrape and save car related data:

1st step) Get all the links for each car advertisements: in this part we can get the car make and model (maybe price too) for each link.

2nd step) Get all car reflated data for each car based on the links that we collected in step 1.

As I told you before I am planing to publish this in two parts. In the first part i will show you the 1st step then in the 2nd part i will show you how to do the 2nd step.

## Go to the landing page of mobile.de
Luckily mobile.de has an English UI too so we do not need to translate all the things to english and we can grab the English elements.

Lets look how we can use Beautifulsoup to get the content of the page:

In [3]:
starting_link_to_scrape = "https://www.mobile.de/?lang=en"
driver.get(starting_link_to_scrape)
time.sleep(1)
base_source = driver.page_source
base_soup = BeautifulSoup(base_source, 'html.parser')

The content of the whole landing page:

In [4]:
#base_soup

First we would need to figure out how to scrape all car's data. Unfortunately if we just list all the cars on the site we won't be able to assign the make and the brand to cars (or I was not able to figure it how to do that) so we need to look for another method: We need to manually grab all car make and model combination and crate a URL based on them.

In [5]:
make_list = base_soup.findAll('div', {'class': 'form-group'})[0]
make_list

<div class="form-group"><label for="qsmakeBuy">Make</label><select "="" class="form-control form-control--dropdown form-control--m mmh-make-incl" id="qsmakeBuy" name="mk"><option selected="selected" value="">Any</option><option class="pmak" value="17200">Mercedes-Benz</option><option class="pmak" value="25200">Volkswagen</option><option class="pmak" value="3500">BMW</option><option class="pmak" value="1900">Audi</option><option disabled="disabled"></option><option value="140">Abarth</option><option value="203">AC</option><option value="375">Acura</option><option value="800">Aixam</option><option value="900">Alfa Romeo</option><option value="1100">ALPINA</option><option value="121">Artega</option><option value="1750">Asia Motors</option><option value="1700">Aston Martin</option><option value="1900">Audi</option><option value="2000">Austin</option><option value="1950">Austin Healey</option><option value="31863">BAIC</option><option value="3100">Bentley</option><option value="3500">BMW</o

In [6]:
one_make = make_list.findAll('option')

In [7]:
car_make = []
id1 = []

for i in range(len(one_make)):
    #print(one_make[i].text.strip())
    car_make.append(one_make[i].text.strip())
    try:
        #print(one_make[i]['value'])
        id1.append(one_make[i]['value'])
    except:
        #print('')
        id1.append('')

car_base_make_data = pd.DataFrame({
    'car_make' : car_make,
    'id1' : id1
    })

In [8]:
car_base_make_data
print(list(car_base_make_data['car_make']))
print(list(car_base_make_data['id1']))

['Any', 'Mercedes-Benz', 'Volkswagen', 'BMW', 'Audi', '', 'Abarth', 'AC', 'Acura', 'Aixam', 'Alfa Romeo', 'ALPINA', 'Artega', 'Asia Motors', 'Aston Martin', 'Audi', 'Austin', 'Austin Healey', 'BAIC', 'Bentley', 'BMW', 'Borgward', 'Brilliance', 'Bugatti', 'Buick', 'Cadillac', 'Casalini', 'Caterham', 'Chatenet', 'Chevrolet', 'Chrysler', 'Citroën', 'Cobra', 'Corvette', 'Cupra', 'Dacia', 'Daewoo', 'Daihatsu', 'DeTomaso', 'DFSK', 'Dodge', 'Donkervoort', 'DS Automobiles', 'Ferrari', 'Fiat', 'Fisker', 'Ford', 'GAC Gonow', 'Gemballa', 'GMC', 'Grecav', 'Hamann', 'Holden', 'Honda', 'Hummer', 'Hyundai', 'Infiniti', 'Isuzu', 'Iveco', 'Jaguar', 'Jeep', 'Kia', 'Koenigsegg', 'KTM', 'Lada', 'Lamborghini', 'Lancia', 'Land Rover', 'Landwind', 'Lexus', 'Ligier', 'Lincoln', 'Lotus', 'Mahindra', 'Maserati', 'Maybach', 'Mazda', 'McLaren', 'Mercedes-Benz', 'MG', 'Microcar', 'MINI', 'Mitsubishi', 'Morgan', 'Nissan', 'NSU', 'Oldsmobile', 'Opel', 'Pagani', 'Peugeot', 'Piaggio', 'Plymouth', 'Polestar', 'Pontiac'

In [9]:
car_make_filter_out = ['Any', 'Other', '']
car_base_make_data = car_base_make_data[~car_base_make_data.car_make.isin(car_make_filter_out)]
car_base_brand_data = car_base_brand_data.drop_duplicates()
car_base_make_data

,car_make,id1
1,Mercedes-Benz,17200
2,Volkswagen,25200
3,BMW,3500
4,Audi,1900
6,Abarth,140
...,...,...
118,Volkswagen,25200
119,Volvo,25100
120,Wartburg,25300
121,Westfield,113


In [21]:
car_base_brand_data = pd.DataFrame()

for one_make in tqdm(car_base_make_data['car_make'], "Progress: "):

    make_string = "//select[@name='mk']/option[text()='{}']".format(one_make)
    driver.find_element_by_xpath(make_string).click()
    time.sleep(3)

    base_source = driver.page_source
    base_soup = BeautifulSoup(base_source, 'html.parser')

    model_list = base_soup.findAll('div', {'class': 'form-group'})[1]
    models = model_list.findAll('option')

    car_model = []
    id2 = []

    for i in range(len(models)):
        
        #print(car_model.append(models[i].text.strip()))
        car_model.append(models[i].text.strip())
        
        try:
            #print(models[i]['value'])
            id2.append(models[i]['value'])
        except:
            #print('')
            id2.append('')

    car_base_model_data = pd.DataFrame({'car_model' : car_model, 'id2' : id2})
    car_base_model_data['car_make'] = one_make

    car_base_brand_data = pd.concat([car_base_brand_data, car_base_model_data], ignore_index=True)
    

Progress: 100%|██████████| 121/121 [06:35<00:00,  3.26s/it]


In [ ]:
car_base_brand_data = car_base_brand_data.drop_duplicates()

In [22]:
car_base_brand_data[car_base_brand_data['car_make'] == "Austin"]

,car_model,id2,car_make
796,Any,,Austin
797,,,Austin
798,Other,1,Austin


In [60]:
car_data_base = pd.merge(car_base_make_data, car_base_brand_data, left_on= ['car_make'], right_on=['car_make'], how = 'right')
car_data_base = car_data_base[~car_data_base.id2.isin([""])]
car_data_base = car_data_base[car_data_base.id2.apply(lambda x: x.isnumeric())]
car_data_base['link'] = "https://suchen.mobile.de/fahrzeuge/search.html?dam=0&isSearchRequest=true&ms=" + car_data_base['id1'] + ";" + car_data_base['id2'] + "&ref=quickSearch&sfmr=false&vc=Car"
car_data_base = car_data_base.reset_index(drop = True)
car_data_base

,car_make,id1,car_model,id2,link
0,Mercedes-Benz,17200,190,126,https://suchen.mobile.de/fahrzeuge/search.html...
1,Mercedes-Benz,17200,190,126,https://suchen.mobile.de/fahrzeuge/search.html...
2,Mercedes-Benz,17200,200,127,https://suchen.mobile.de/fahrzeuge/search.html...
3,Mercedes-Benz,17200,200,127,https://suchen.mobile.de/fahrzeuge/search.html...
4,Mercedes-Benz,17200,220,128,https://suchen.mobile.de/fahrzeuge/search.html...
...,...,...,...,...,...
3985,Wiesmann,25650,MF 30,4,https://suchen.mobile.de/fahrzeuge/search.html...
3986,Wiesmann,25650,MF 35,6,https://suchen.mobile.de/fahrzeuge/search.html...
3987,Wiesmann,25650,MF 4,7,https://suchen.mobile.de/fahrzeuge/search.html...
3988,Wiesmann,25650,MF 5,8,https://suchen.mobile.de/fahrzeuge/search.html...


In [65]:
random_number = randrange(len(car_data_base))
print("make: ", car_data_base['car_make'][random_number], " model: ", car_data_base['car_model'][random_number])
print(car_data_base['link'][random_number])

make:  Mercedes-Benz  model:  ML 420
https://suchen.mobile.de/fahrzeuge/search.html?dam=0&isSearchRequest=true&ms=17200;192&ref=quickSearch&sfmr=false&vc=Car


# Save CSV

In [66]:
car_data_base.to_csv("car_data_base.csv", encoding='utf-8', index=False)

# Put everything together into one function

In [4]:
def get_all_make_model(mobile_de_eng_base_link="https://www.mobile.de/?lang=en", save_filename="make_and_model_links.csv"):

    from selenium import webdriver
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.chrome.options import Options
    from bs4 import BeautifulSoup
    import pandas as pd
    import numpy as np
    from random import randrange
    from tqdm import tqdm
    import time

    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)

    driver.get(mobile_de_eng_base_link)
    time.sleep(3)
    base_source = driver.page_source
    base_soup = BeautifulSoup(base_source, 'html.parser')

    make_list = base_soup.findAll('div', {'class': 'form-group'})[0]
    one_make = make_list.findAll('option')

    car_make = []
    id1 = []

    for i in range(len(one_make)):

        car_make.append(one_make[i].text.strip())

        try:
            id1.append(one_make[i]['value'])
        except:
            id1.append('')

    car_base_make_data = pd.DataFrame({'car_make': car_make, 'id1': id1})

    car_make_filter_out = ['Any', 'Other', '']
    car_base_make_data = car_base_make_data[~car_base_make_data.car_make.isin(car_make_filter_out)]
    car_base_make_data = car_base_make_data.drop_duplicates()
    car_base_make_data = car_base_make_data.reset_index(drop=True)

    car_base_brand_data = pd.DataFrame()

    for one_make in tqdm(car_base_make_data['car_make'], "Progress: "):

        make_string = "//select[@name='mk']/option[text()='{}']".format(one_make)
        driver.find_element_by_xpath(make_string).click()
        time.sleep(3)

        base_source = driver.page_source
        base_soup = BeautifulSoup(base_source, 'html.parser')

        model_list = base_soup.findAll('div', {'class': 'form-group'})[1]
        models = model_list.findAll('option')

        car_model = []
        id2 = []

        for i in range(len(models)):

            car_model.append(models[i].text.strip())

            try:
                id2.append(models[i]['value'])
            except:
                id2.append('')

        car_base_model_data = pd.DataFrame({'car_model': car_model, 'id2': id2})
        car_base_model_data['car_make'] = one_make

        car_base_brand_data = pd.concat(
            [car_base_brand_data, car_base_model_data], ignore_index=True)

    car_data_base = pd.merge(car_base_make_data, car_base_brand_data, left_on=['car_make'], right_on=['car_make'], how='right')
    car_data_base = car_data_base[~car_data_base.id2.isin([""])]
    car_data_base = car_data_base[car_data_base.id2.apply(lambda x: x.isnumeric())]
    car_data_base = car_data_base.drop_duplicates()
    
    car_data_base['link'] = "https://suchen.mobile.de/fahrzeuge/search.html?dam=0&isSearchRequest=true&ms=" + car_data_base['id1'] + ";" + car_data_base['id2'] +  "&ref=quickSearch&sfmr=false&vc=Car"
    car_data_base = car_data_base.reset_index(drop=True)

    if len(save_filename) > 0:
        car_data_base.to_csv(save_filename, encoding='utf-8', index=False)

    return(car_data_base)


In [5]:
all_data = get_all_make_model(save_filename="make_and_model_links.csv")



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-4-29fbccade488>:17: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
Progress: 100%|██████████| 117/117 [06:20<00:00,  3.25s/it]


In [3]:
all_data

,car_make,id1,car_model,id2,link
0,Mercedes-Benz,17200,190,126,https://suchen.mobile.de/fahrzeuge/search.html...
1,Mercedes-Benz,17200,200,127,https://suchen.mobile.de/fahrzeuge/search.html...
2,Mercedes-Benz,17200,220,128,https://suchen.mobile.de/fahrzeuge/search.html...
3,Mercedes-Benz,17200,230,129,https://suchen.mobile.de/fahrzeuge/search.html...
4,Mercedes-Benz,17200,240,130,https://suchen.mobile.de/fahrzeuge/search.html...
...,...,...,...,...,...
2230,Wiesmann,25650,MF 30,4,https://suchen.mobile.de/fahrzeuge/search.html...
2231,Wiesmann,25650,MF 35,6,https://suchen.mobile.de/fahrzeuge/search.html...
2232,Wiesmann,25650,MF 4,7,https://suchen.mobile.de/fahrzeuge/search.html...
2233,Wiesmann,25650,MF 5,8,https://suchen.mobile.de/fahrzeuge/search.html...
